In [1]:
# coding: utf-8
import numpy as np
import tensorflow as tf

In [2]:
C = 10 # classes num
N = 784 # features num
learning_rate = 0.01
epochs = 2
batch_size = 128
num_steps = 2000
display_step = 100

In [3]:
def model_net(X, n_classes, n_features):
    W = tf.Variable(tf.zeros([n_features, n_classes]), name='W')
    b = tf.Variable(tf.zeros(n_classes), name='b')

    logits = tf.matmul(X, W) + b
    return logits

In [4]:
def log_loss_fn(logits, labels, l2_reg=0):
    logits_loss = tf.losses.sparse_softmax_cross_entropy(labels, logits=logits)
    W = tf.get_default_graph().get_tensor_by_name('W:0')
    l2_loss = l2_reg * tf.nn.l2_loss(W)
    return logits_loss + l2_loss, logits_loss, l2_loss

In [14]:
def hinge_loss_fn(logits, labels, l2_reg=0):
#     logits_loss = tf.losses.hinge_loss(labels, logits=logits)
    labels = (tf.one_hot(labels, C) - 0.5) * 2
    logits_loss = tf.reduce_mean(tf.maximum(0.0, 1 - logits * labels))
    W = tf.get_default_graph().get_tensor_by_name('W:0')
    l2_loss = l2_reg * tf.nn.l2_loss(W)
    return logits_loss + l2_loss, logits_loss, l2_loss

In [6]:
def accuracy_fn(logits, labels):
    predictions = tf.nn.softmax(logits)
    is_correct = tf.equal(tf.argmax(predictions, axis=1), labels)
    accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
    return accuracy

In [7]:
def train_input_fn(features, labels, batch_size, shuffle_size=1000):
    dataset = tf.data.Dataset.from_tensor_slices((features, labels))
    dataset = dataset.shuffle(shuffle_size).repeat().batch(batch_size).prefetch(batch_size)
    return dataset.make_one_shot_iterator().get_next()

In [8]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=False)
x_train, y_train = mnist.train.images, mnist.train.labels

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [9]:
class CumulateAvg:
    def __init__(self, var_list, display_step=100):
        self.var_list = var_list
        self.reset()
        self.display_step = display_step
        self.step = 0
        
    def reset(self):
        self.var_dict = {k: 0 for k in self.var_list}
        
    def avg_var(self):
        if self.step > 0:
            self.var_dict = {k: v / self.display_step for k, v in self.var_dict.items()}
        
    def cumulate(self, var_values):
        for i, v in enumerate(var_values):
            self.var_dict[self.var_list[i]] += v
        if self.step == 0 or (self.step + 1) % self.display_step == 0:
            self.avg_var()
            print_str = 'step: %d' % (self.step + 1)
            print_str += ''.join([', %s: %.4f' % (k, self.var_dict[k]) for k in self.var_list])
            print(print_str)
        self.step += 1

In [15]:
with tf.Graph().as_default() as g:
    with tf.Session().as_default() as sess:
        dataset = train_input_fn(x_train, y_train, batch_size)
        X, y = dataset
        X = tf.cast(X, tf.float32)
        y = tf.cast(y, tf.int64)
        
        logits = model_net(X, C, N)
#         loss, logits_loss, l2_loss = log_loss_fn(logits, y, l2_reg=0.05)
        loss, logits_loss, l2_loss = hinge_loss_fn(logits, y, l2_reg=0.05)
#         print(loss, logits_loss, l2_loss)
        accuracy = accuracy_fn(logits, y)
        train_op = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
        sess.run(tf.global_variables_initializer())

        var_list = ['loss', 'accuracy', 'logits_loss', 'l2_loss']
        cumavg_obj = CumulateAvg(var_list, 100)
        for step in range(num_steps):
            res_list = sess.run([train_op, *[eval(x) for x in var_list]])
            cumavg_obj.cumulate(res_list[1:])

Tensor("add_1:0", shape=(), dtype=float32) Tensor("Mean:0", shape=(), dtype=float32) Tensor("mul_2:0", shape=(), dtype=float32)
step: 1, loss: 1.0000, accuracy: 0.1172, logits_loss: 1.0000, l2_loss: 0.0000
step: 100, loss: 0.3918, accuracy: 0.6298, logits_loss: 0.3855, l2_loss: 0.0063
step: 200, loss: 0.2536, accuracy: 0.6739, logits_loss: 0.2422, l2_loss: 0.0114
step: 300, loss: 0.2441, accuracy: 0.6970, logits_loss: 0.2310, l2_loss: 0.0131
step: 400, loss: 0.2332, accuracy: 0.7213, logits_loss: 0.2191, l2_loss: 0.0141
step: 500, loss: 0.2280, accuracy: 0.7357, logits_loss: 0.2129, l2_loss: 0.0152
step: 600, loss: 0.2254, accuracy: 0.7272, logits_loss: 0.2091, l2_loss: 0.0164
step: 700, loss: 0.2247, accuracy: 0.7359, logits_loss: 0.2068, l2_loss: 0.0178
step: 800, loss: 0.2192, accuracy: 0.7433, logits_loss: 0.2001, l2_loss: 0.0191
step: 900, loss: 0.2134, accuracy: 0.7760, logits_loss: 0.1932, l2_loss: 0.0202
step: 1000, loss: 0.2153, accuracy: 0.7482, logits_loss: 0.1940, l2_loss: 